# Steps for next iteration of this notebook

* Fit func gen to get base freq.
* Fit n-harmonics of of both resistor integral and secondary to get fit functions
* Plot fit results against data to ensure accuracy
* Think about physics of how these two voltages should be related to express in B/H
* Maybe make a harmonic result class that has methods evaluate, derivative, integral
* Pick a mid point in time range and fmin to different points on graph to find
  - remanence
  - coercivity
  - initial permeability


In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
from daq.pico import CSV
from scipy.optimize import fmin
from easier import ParamState, shade, Item
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, Ridge
hv.extension('bokeh')

In [ ]:
def get_freq(df, freq_guess):
    x = df.t.values
    y = df.sig_gen.values

    p = ParamState(
        'x',
        'y_true',
        a=1,
        f=freq_guess,
        phi=0
    )
    p.given(
        x=x,
        y_true=y
    )

    def model(p):
        return p.a * np.sin(2 * np.pi * p.f * x + p.phi)

    def cost(args, p):
        p.ingest(args)
        err = model(p) - p.y_true
        return np.sum(err ** 2)

    x0 = p.array
    xf = fmin(cost, x0, args=(p,), disp=False)
    p.ingest(xf)
    return p.f

In [ ]:
file_name = './data_20171220/20171220-0003.csv'
# file_name = './data_20171220/20171220-0001.csv'  # air

freq_guess = 20
df = CSV(file_name, a='sig_gen', b='res_volt', c='sec_volt', max_sample_freq=3000000).df
# df = df.head(1000)
# f0 = get_freq(df, freq_guess)
# max_harmonic = 4

# df_basis = pd.DataFrame(index=df.index)

# for n in range(1, max_harmonic + 1):
#     f = n * f0
#     w = 2 * np.pi * f
#     scol = f'sin{n:02d}'
#     ccol = f'cos{n:02d}'
#     df_basis.loc[:, scol] = np.sin(n * w * df.t)
#     df_basis.loc[:, ccol] = np.sin(n * w * df.t)
df.head()

In [ ]:
def get_basis_frame(df, fundamental_freq, num_freqs):
    f0 = get_freq(df, fundamental_freq)
    df_basis = pd.DataFrame(index=df.index)

    for n in range(1, num_freqs + 1):
        f = n * f0
        w = 2 * np.pi * f
        scol = f'sin{n:02d}'
        ccol = f'cos{n:02d}'
        df_basis.loc[:, scol] = np.sin(n * w * df.t)
        df_basis.loc[:, ccol] = np.cos(n * w * df.t)
    return Item(
        df=df_basis,
        f0=f0,
        num_freqs=num_freqs,
        w0=2 * np.pi * f0
    )
    


In [ ]:
def run_fit(df, fundamental_freq, num_freqs):
    basis = get_basis_frame(df, fundamental_freq, num_freqs)
    df_basis = basis.df
    
    y_res = df.res_volt
    y_sec = df.sec_volt

#     model = LinearRegression()
    # model = RidgeCV(store_cv_values=True)
    # model = LassoCV()
    alpha = .000002
    
    model_res = Ridge(alpha=alpha)
    model_res.fit(df_basis.values, y_res.values)
#     print(model_res.coef_)
    
    model_sec = Ridge(alpha=alpha)
    model_sec.fit(df_basis.values, y_sec.values)
#     print(model_sec.coef_)
    
    y_res_fit = model_res.predict(df_basis.values)
    y_sec_fit = model_sec.predict(df_basis.values)
    
    return Item(
        res_fit=y_res_fit,
        sec_fit=y_sec_fit,
        res_coeffs=model_res.coef_,
        sec_coeffs=model_sec.coef_,
        basis=basis,
        df=df,
    )
    return y_res_fit, y_sec_fit, model_res.coef_

file_name = './data_20171220/20171220-0003.csv'
fundamental_freq = 20
num_freqs = 5
res = run_fit(df, fundamental_freq, num_freqs)
item = res
res


In [ ]:
def make_fit_functions(item):
    terms = {
        'sec_fit': [],
        'res_int_fit': []
    }
    f0 = item.basis.f0
    num_freqs = item.basis.num_freqs
    for n in range(1, num_freqs + 1):
        w = 2 * np.pi * f
        sfunc = lambda t: np.sin(n * w * t)
        cfunc = lambda t: np.cos(n * w * t)
        terms['']


    

    

In [ ]:
def integrate(item, coeffs='res'):
    f0 = item.basis.f0
    num_freqs = item.basis.num_freqs
    df_integral = pd.DataFrame(index=item.basis.df.index)

    for n in range(1, num_freqs + 1):
        f = n * f0
        w = 2 * np.pi * f
        scol = f'sin{n:02d}'
        ccol = f'cos{n:02d}'
        # these are the integrals of the named columns
        df_integral.loc[:, scol] = - np.cos(n * w * df.t) / (n * w)
        df_integral.loc[:, ccol] = np.sin(n * w * df.t) / (n * w)
    df_integral = df_integral * item[coeffs + '_coeffs']
    integral = df_integral.sum(axis=1)
    item[f'{coeffs}_integral_fit'] = integral
    
    return item
                                    

item = integrate(item)    
item

In [ ]:
item.df.head()

In [ ]:
t = item.df.t
(
    shade(hv.Curve((t, item.res_fit)), color='blue')
    * shade(hv.Curve((t, item.res_integral_fit.diff() / t.diff().median())), color='red')
)

In [ ]:
%opts RGB [width=400, height=400]
shade(hv.Curve((item.res_integral_fit, item.sec_fit)))